In [ ]:
# default_exp secret
# hide

_FNAME='secret'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os

TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
HERE = os.path.abspath('')

In [ ]:
#export
import boto3
CLIENT = boto3.client('secretsmanager')

In [ ]:
import ichorlib.testtools as TT
import localstack_client.session

def run_localstack(version="latest", **kwargs):
    env = {'SERVICES':'secretsmanager'}
    additional_kwargs = {'environment': env}
    additional_kwargs.update(kwargs)
    ports = {4566:4566}
    image = 'localstack/localstack:{}'.format(version)
    ls_container = TT.run_container(image, ports=ports, block_until_port_available=4566,
                                   **additional_kwargs)
    ls_container = TT.set_port_name(ls_container, 4566, 'entry')
    return ls_container    

def make_local_client():
    session = localstack_client.session.Session()
    return session.client('secretsmanager')

CONTAINER_NAME = 'localstack-vera-secret'

In [ ]:
#export
def get(name, client=None, **kwargs):
    '''
    Retrieves the secret with the given name
    '''
    client = client or CLIENT
    resp = client.get_secret_value(SecretId=name, **kwargs)
    str_value = resp.get('SecretString')
    byte_value = resp.get('SecretBytes')
    return str_value or byte_value

def create(name, value, client=None, **kwargs):
    '''
    Create a new secret with Name=name
    If you want SecretBytes instead of SecretString
    set value=None and specify SecretBytes in kwargs
    '''
    client = client or CLIENT
    if value is not None:
        kwargs['SecretString'] = value
    
    resp = client.create_secret(Name=name, **kwargs)
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

def update(name, value, client=None, **kwargs):
    '''
    Update the value of a secret with SecretId=name
    '''
    client = client or CLIENT
    if value is not None:
        kwargs['SecretString'] = value
    
    resp = client.put_secret_value(SecretId=name, **kwargs)    
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

def delete(name, client=None, **kwargs):
    '''
    Schedule the secret with SecretId=name for deletion
    '''
    client = client or CLIENT
    resp = client.delete_secret(SecretId=name, **kwargs)    
    ok = resp['ResponseMetadata']['HTTPStatusCode']==200
    return ok, resp

Here is a non-automated test that engages the live aws service.

In [ ]:
import uuid
def LIVE_test():
    ls_container = run_localstack(container_name=CONTAINER_NAME)
    client = make_local_client()
    
    secret_name = 'unittest/{}'.format(uuid.uuid4())

    ok, resp = create(secret_name, 'testvalue', client=client)
    #print(ok, resp)
    TESTCASE.assertTrue(ok)

    TESTCASE.assertEqual( get(secret_name, client=client), 'testvalue')

    ok, resp = update(secret_name, 'newvalue', client=client)
    #print(ok, resp)
    TESTCASE.assertTrue(ok)
    TESTCASE.assertEqual( get(secret_name, client=client), 'newvalue')

    ok, resp = delete(secret_name, client=client)
    #print(ok, resp)

    with TESTCASE.assertRaises(CLIENT.exceptions.InvalidRequestException):
        get(secret_name, client=client)
        
LIVE_test()

NameError: name 'CONTAINER_NAME' is not defined

In [ ]:
notebook2script(_nbpath)

Converted secret.ipynb.
